Szukam chętnych do obliczenia metodą Kontka liczby outlierów w innych wyborach (np. 2023, 2019, 2015, 2010, 2007) oraz do obliczenia odchyleń przestrzennych dla 2025 i poprzednich wyborów w sensowniejszy sposób. Myślałem o czymś takim: trzeba (1) zrobić geokodowanie obwodów (trochę mamy zrobione, ale tylko trochę) i (2) policzyć dla każdego obwodu, czy jest on outlierem od klastra $k$ okolicznych obwodów, gdzie $k$ jest do ustalenia (może 20?).

1) load and clean data (2015, 2020, 2025)
2) cluster by Kontek, but should easily be replaced
3) use Kontek methods
4) obliczyć odchylenia przestrzenne

In [15]:
import pandas as pd
import re
import os
from collections import defaultdict
import numpy as np

In [16]:
POLAND_RAW_DATA = os.path.join(os.getcwd(), "data", "raw", "poland")
POLAND_RAW_DATA

'/Users/gignac/Desktop/Projects/electoral-anomalies/data/raw/poland'

### PRESIDENTIAL

In [17]:
BASE_COLUMNS_MAP = {
    # 2025
    "Nr komisji": "polling_station_id",
    "Teryt Gminy": "teryt_gmina",
    # "Teryt Powiatu": "teryt_powiat", 
    "Liczba wyborców uprawnionych do głosowania (umieszczonych w spisie, z uwzględnieniem dodatkowych formularzy) w chwili zakończenia głosowania": "eligible_voters",
    "Liczba kart wyjętych z urny": "ballots_cast",
    # round 2
    "Liczba głosów ważnych oddanych łącznie na wszystkich kandydatów (z kart ważnych)": "valid_votes",
    # round 1
    "Liczba głosów ważnych oddanych łącznie na obu kandydatów (z kart ważnych)": "valid_votes",
    # 2020
    "Numer obwodu": "polling_station_id",
    "Kod TERYT": "teryt_gmina",
    "Liczba wyborców uprawnionych do głosowania": "eligible_voters",
    "Liczba kart wyjętych z urny": "ballots_cast",
    "Liczba głosów ważnych oddanych łącznie na wszystkich kandydatów": "valid_votes",
    # 2015
    "Liczba głosów ważnych": "valid_votes",
}

In [18]:
CANDIDATE_SURNAMES_BY_YEAR = {
    "2015": [
        "braun", "duda", "jarubas", "komorowski", "korwin-mikke",
        "kowalski", "kukiz", "ogórek", "palikot", "tanajno", "wilk"
    ],
    "2020": [
        "biedroń", "bosak", "duda", "hołownia", "jakubiak",
        "kosiniak-kamysz", "piotrowski", "tanajno", "trzaskowski",
        "witkowski", "żółtek"
    ],
    "2025": [
        "bartoszewicz", "biejat", "braun", "hołownia", "jakubiak",
        "maciak", "mentzen", "nawrocki", "senyszyn", "stanowski",
        "trzaskowski", "woch", "zandberg"
    ]
}


def get_candidate_rename_map(df, year):
    surnames = CANDIDATE_SURNAMES_BY_YEAR.get(str(year), [])
    rename_map = {
        col: surname
        for col in df.columns
        for surname in surnames
        if surname in col.lower()
    }
    return rename_map

In [19]:
def extract_postal_code(address):
    match = re.search(r"\b\d{2}-\d{3}\b", str(address))
    return match.group(0) if match else None

In [ ]:
def load_presidential_data(year, round, ext="csv"):
    file_path = os.path.join(POLAND_RAW_DATA, f"{year}_presidential", f"round{round}.{ext}")
    if ext == "csv":
        df = pd.read_csv(file_path, sep=";", encoding="utf-8")
    elif ext == "xls":
        df = pd.read_excel(file_path, dtype={"TERYT gminy": str})
    else:
        raise NotImplementedError(f"Cannot load file with {ext} ext!")
    # Normalize column names: replace non-breaking spaces, strip whitespace
    df.columns = df.columns.str.replace("\xa0", " ", regex=False).str.strip()
    return df

In [271]:
def process_presidential_df(df, year, final_cols=[]):
    # Rename columns - to have same naming convention for all files
    df = df.rename(columns=BASE_COLUMNS_MAP)

    # Shorten candidate columns - risky if two candidates with the same surname

    candidate_rename_map = get_candidate_rename_map(df, year=year)
    df.rename(columns=candidate_rename_map, inplace=True)
    
    df = df.rename(columns=candidate_rename_map)
    # Assuming, if postal code column exits, no need for extraction
    if "postal_code" not in df.columns:
        # Get postal code from address ("siedziba")
        df["postal_code"] = df["Siedziba"].apply(extract_postal_code)
    final_cols = (
        list(set(BASE_COLUMNS_MAP.values()))  # ensure uniqueness only here
        + list(candidate_rename_map.values())  # may have overlaps, OK
        + ["postal_code"] + final_cols
    )
    df = df[final_cols]
    return df

In [22]:
def get_presidential_df(year, round, ext="csv"):
    df = load_presidential_data(year, round, ext)
    return process_presidential_df

In [135]:
year = "2025"
df_2025_r1 = process_presidential_df(load_presidential_data(year, "1"), year)
df_2025_r2 = process_presidential_df(load_presidential_data(year, "2"), year)
year = "2020"
df_2020_r1 = process_presidential_df(load_presidential_data(year, "1"), year)
df_2020_r2 = process_presidential_df(load_presidential_data(year, "2"), year)
year = "2015"
ext = "xls"
df_2015_r1 = process_presidential_df(load_presidential_data(year, "1", ext), year)

# In 2015 round 2 data there is no postal code, so we need to join it to round 1
df_2015_r2 = load_presidential_data(year, "2", ext)
df_2015_r2 = df_2015_r2.merge(
    df_2015_r1[["teryt_gmina", "postal_code"]],
    left_on="Teryt Gminy",
    right_on="teryt_gmina",
    how="left"
)
df_2015_r2 = process_presidential_df(df_2015_r2, year)

/var/folders/27/sdwyh2gs5mx77yx8h4jgxnjw0000gn/T/ipykernel_55677/4050919450.py:4: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=";", encoding="utf-8")
/var/folders/27/sdwyh2gs5mx77yx8h4jgxnjw0000gn/T/ipykernel_55677/4050919450.py:4: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=";", encoding="utf-8")


### 2.2. Grupowanie geograficzne

W drugim etapie komisje wyborcze zostały posortowane według kodów pocztowych, a następnie
pogrupowane w kolejne bloki komisji znajdujących się w bezpośrednim sąsiedztwie. Grupy tworzono
w taki sposób, aby — w miarę możliwości — każda zawierała od 10 do 16 komisji, łącząc ze sobą
sąsiednie obszary kodów pocztowych mające wspólny prefiks (np. „30”, „301”, „3011”). Celem było
maksymalne zwiększenie spójności przestrzennej przy zachowaniu poręcznej wielkości grupy, bez
konieczności ograniczania jej do jednego kodu pocztowego. Zastosowano następującą procedurę:

  1. Początkowe grupowanie oparto na pierwszych dwóch cyfrach kodu pocztowego (np. „30” dla
obszaru Krakowa).
  2. Jeżeli powstała grupa zawierała od 10 do 16 komisji, została zaakceptowana bez zmian.
  3. Grupy liczące mniej niż 10 komisji odłożono do późniejszego łączenia.
  4. Grupy przekraczające 16 komisji dzielono rekurencyjnie, dodając kolejne cyfry kodu pocztowego (np. z „30” → „301” → „3011” i dalej, aż do pełnych pięciu cyfr).
  5. Pozostałe małe grupy łączono z najbliższymi sąsiadami mającymi ten sam prefiks, przy czym priorytetem była ciągłość przestrzenna i zrównoważona liczebność grup.
  
W odróżnieniu od wcześniejszego podejścia, które dopuszczało grupy o wielkości 10–25 komisji,
niniejsze badanie przyjęło węższy zakres docelowy: od 10 do 16 komisji na grupę. Decyzja ta
wynikała z przeglądu empirycznego, który wykazał, że większe grupy — mimo wydajności
statystycznej — czasami łączyły odległe geograficznie obszary o niejednorodnych wzorcach
głosowania.

W wyniku zastosowania nowych ograniczeń utworzono 2 208 grup, z których każda odzwierciedlała
względnie jednorodną lokalną dynamikę wyborczą. Dla potwierdzenia ich spójności terytorialnej
przeprowadzono test zgodności kodów pocztowych w ramach każdej grupy.

Większość grup spełniła założony docelowy rozmiar: 1 386 grup (62,8%) zawierało od 10 do 16
komisji, a 2 017 grup (91,3%) zawierało od 6 do 30 komisji. Większe grupy zazwyczaj odpowiadały obszarom miejskim — na przykład takim jak Toruń czy Włocławek, gdzie pojedynczy kod pocztowy
obejmował całe miasto. W takich przypadkach większa liczba komisji nie zaburzała spójności
przestrzennej, a wręcz zwiększała wiarygodność statystyczną poprzez powiększenie próbki lokalnej.

Grupy mniejsze niż docelowy zakres obejmowały komisje, które — z powodu izolacji geograficznej
— nie mogły zostać sensownie połączone z innymi. Choć próbki mniejsze niż 10 jednostek są zwykle
uznawane za mające ograniczoną moc statystyczną, zastosowanie metody MAD — znanej ze swojej
odporności na małe próby — w znacznym stopniu niweluje to ograniczenie.

------------

Nie jestem w stanie odtworzyć grupowania geograficznego 1:1.

Jedną z opcji jest grupowanie losowe: buckety 10 - 16, dokladnie 2208 grup

In [24]:
def add_random_buckets(df, n, k_min, k_max, random_state=None):
    """
    Partition a DataFrame into n random groups (buckets), each with at least k_min and at most k_max rows.
    Adds a 'bucket' column to the DataFrame indicating the group assignment.

    Args:
        df (pd.DataFrame): The DataFrame to partition.
        n (int): Number of buckets (groups) to create.
        k_min (int): Minimum number of rows per bucket.
        k_max (int): Maximum number of rows per bucket.
        random_state (int, optional): Seed for reproducibility. Default is None.

    Returns:
        pd.DataFrame: A new DataFrame with a 'bucket' column indicating the group assignment.

    Raises:
        ValueError: If the constraints cannot be satisfied with the given n, k_min, and k_max.
    """
    N = len(df)
    if N < n * k_min or N > n * k_max:
        raise ValueError("Constraints cannot be satisfied with given n, k_min, and k_max.")
    
    # Start with k_min in each bucket
    sizes = np.full(n, k_min)
    remaining = N - sizes.sum()
    increments = np.zeros(n, dtype=int)
    
    rng = np.random.default_rng(random_state)
    while remaining > 0:
        idxs = np.where(sizes + increments < k_max)[0]
        idx = rng.choice(idxs)
        increments[idx] += 1
        remaining -= 1
    
    final_sizes = sizes + increments
    
    # Shuffle the DataFrame
    shuffled_df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    # Assign bucket numbers
    bucket_labels = np.repeat(np.arange(n), final_sizes)
    shuffled_df['bucket'] = bucket_labels
    
    # Restore original order if needed (optional)
    # shuffled_df = shuffled_df.sort_index()
    
    return shuffled_df

In [25]:
df = add_random_buckets(df_2025_r2, n=2208, k_min=10, k_max=16)

In [26]:
def print_bucket_stats(df, bucket_col='bucket', range1=(10, 16), range2=(6, 30)):
    """
    Prints statistics about bucket sizes in the DataFrame.
    Shows total number of buckets and how many fall into two specified ranges.

    Args:
        df (pd.DataFrame): DataFrame containing the bucket column.
        bucket_col (str): Name of the column with bucket assignments.
        range1 (tuple): First (min, max) range for bucket size.
        range2 (tuple): Second (min, max) range for bucket size.
    """
    bucket_sizes = df[bucket_col].value_counts()
    total_buckets = bucket_sizes.count()
    buckets_in_range1 = bucket_sizes[(bucket_sizes >= range1[0]) & (bucket_sizes <= range1[1])].count()
    buckets_in_range2 = bucket_sizes[(bucket_sizes >= range2[0]) & (bucket_sizes <= range2[1])].count()
    print(f"Total buckets: {total_buckets}")
    print(f"Buckets with {range1[0]}–{range1[1]} items: {buckets_in_range1} ({buckets_in_range1 / total_buckets:.1%})")
    print(f"Buckets with {range2[0]}–{range2[1]} items: {buckets_in_range2} ({buckets_in_range2 / total_buckets:.1%})")


In [27]:
print_bucket_stats(df)

Total buckets: 2208
Buckets with 10–16 items: 2208 (100.0%)
Buckets with 6–30 items: 2208 (100.0%)


Drugą moje "klastrowanie"

In [ ]:
def add_janiszewski_postal_buckets(df, min_bucket_size=10, max_bucket_size=16, postal_code_col='postal_code'):
    """
    Assigns spatially-coherent buckets to a DataFrame based on postal code prefixes, with constraints on bucket size.
    The function mimics the logic from generate_buckets_3.ipynb.

    Args:
        df (pd.DataFrame): Input DataFrame. Must contain a column with postal codes.
        min_bucket_size (int): Minimum number of rows per bucket.
        max_bucket_size (int): Maximum number of rows per bucket.
        postal_code_col (str): Name of the column containing postal codes (e.g., 'postal_code').

    Returns:
        pd.DataFrame: DataFrame with a new 'bucket' column indicating group assignment.
    """
    df = df.copy()
    # Clean postal codes: remove dash, ensure string
    df['postal_clean'] = df[postal_code_col].astype(str).str.replace('-', '')
    
    # Compute postal code prefixes
    df['postal_2'] = df['postal_clean'].str[:2]
    df['postal_3'] = df['postal_clean'].str[:3]
    df['postal_4'] = df['postal_clean'].str[:4]
    df['postal_5'] = df['postal_clean']
    
    # Precompute value counts for each prefix
    df['postal_3_value_count'] = df['postal_3'].map(df['postal_3'].value_counts())
    df['postal_4_value_count'] = df['postal_4'].map(df['postal_4'].value_counts())
    
    # Step 1: Assign buckets by 3-digit prefix if group size is valid
    postal_3_counts = df['postal_3'].value_counts()
    valid_postals_3 = postal_3_counts[(postal_3_counts >= min_bucket_size) & (postal_3_counts <= max_bucket_size)].index
    df['bucket'] = df['postal_3'].where(df['postal_3'].isin(valid_postals_3))
    
    # Step 2: Assign by 4-digit prefix for unassigned rows
    postal_4_counts = df['postal_4'].value_counts()
    valid_postals_4 = postal_4_counts[(postal_4_counts >= min_bucket_size) & (postal_4_counts <= max_bucket_size)].index
    mask_4 = df['bucket'].isna() & df['postal_4'].isin(valid_postals_4)
    df.loc[mask_4, 'bucket'] = df.loc[mask_4, 'postal_4']
    
    # Step 3: Assign by 5-digit (full) postal code for unassigned rows
    postal_5_counts = df['postal_5'].value_counts()
    valid_postals_5 = postal_5_counts[(postal_5_counts >= min_bucket_size) & (postal_5_counts <= max_bucket_size)].index
    mask_5 = df['bucket'].isna() & df['postal_5'].isin(valid_postals_5)
    df.loc[mask_5, 'bucket'] = df.loc[mask_5, 'postal_5']
    
    # Step 4: For oversized 5-digit groups, assign if all are ungrouped
    mask_postal_5 = (
        df['bucket'].isna() &
        ((df['postal_3_value_count'] > max_bucket_size) |
         (df['postal_4_value_count'] > max_bucket_size))
    )
    postal_5_counts = df.loc[mask_postal_5, 'postal_5'].value_counts()
    for postal_code, count in postal_5_counts.items():
        if count > max_bucket_size:
            same_postal_mask = (df['postal_5'] == postal_code) & df['bucket'].isna()
            if same_postal_mask.sum() == count:
                df.loc[same_postal_mask, 'bucket'] = postal_code
    
    # Step 5: For remaining unassigned, chunk by sorted postal code within 3-digit prefix
    leftovers = df[df['bucket'].isna()].copy()
    for prefix, group in leftovers.groupby('postal_3'):
        group_sorted = group.sort_values(by='postal_clean')
        i = 0
        while i < len(group_sorted):
            chunk = group_sorted.iloc[i:i+max_bucket_size]
            if len(chunk) >= min_bucket_size:
                bucket_id = f"{prefix}_L{i//max_bucket_size}"
                df.loc[chunk.index, 'bucket'] = bucket_id
                i += len(chunk)
            else:
                break  # remaining rows too small to form a valid group
    
    # Step 6: Try to merge final leftovers into existing buckets, else assign fallback
    final_leftovers = df[df['bucket'].isna()].copy()
    existing_buckets = df.dropna(subset=['bucket']).copy()
    existing_buckets['bucket_size'] = existing_buckets.groupby('bucket')['bucket'].transform('count')
    for prefix, group in final_leftovers.groupby('postal_3'):
        group_sorted = group.sort_values(by='postal_clean')
        existing_in_prefix = existing_buckets[
            existing_buckets['postal_3'] == prefix
        ].groupby('bucket').first()
        for idx, row in group_sorted.iterrows():
            # Try to append to a not-too-large existing group
            assigned = False
            for bucket_id, b_row in existing_in_prefix.iterrows():
                current_size = df[df['bucket'] == bucket_id].shape[0]
                if current_size < max_bucket_size:
                    df.at[idx, 'bucket'] = bucket_id
                    assigned = True
                    break
            # If no spot found, assign a new fallback bucket
            if pd.isna(df.at[idx, 'bucket']):
                fallback_id = f"{prefix}_F{idx}"
                df.at[idx, 'bucket'] = fallback_id
    
    if df['bucket'].isna().sum() > 0:
        # Find all existing buckets and their sizes
        bucket_sizes = df['bucket'].value_counts()
        smallest_buckets = bucket_sizes.nsmallest(10).index  # or all buckets
        for idx in df[df['bucket'].isna()].index:
            # Assign to the bucket with the smallest current size
            target_bucket = df['bucket'].value_counts().idxmin()
            df.at[idx, 'bucket'] = target_bucket


    # Optionally, remove buckets with fewer than 3 members
    vc = df['bucket'].value_counts()
    df = df[df['bucket'].isin(vc[vc >= 3].index)]

    # Remove all helper columns before returning
    helper_cols = [
        'postal_clean', 'postal_2', 'postal_3', 'postal_4', 'postal_5',
        'postal_3_value_count', 'postal_4_value_count'
    ]
    df = df.drop(columns=[col for col in helper_cols if col in df.columns])
    
    return df


In [ ]:
df = add_janiszewski_postal_buckets(df_2025_r2, min_bucket_size=10, max_bucket_size=16)


In [30]:
print_bucket_stats(df)

Total buckets: 1809
Buckets with 10–16 items: 1462 (80.8%)
Buckets with 6–30 items: 1719 (95.0%)


Trzecią klastrowanie Jakuba Bialka
https://github.com/rabitwhte/analiza_kontka_reprodukcja/blob/main/Reprodukcja_wynikow_Kontek_Bialek.ipynb

In [35]:
def add_bialek_postal_buckets(df: pd.DataFrame,
                  postal_code_col: str = "postal_code",
                  min_size: int = 10,
                  max_size: int = 16) -> pd.DataFrame:
    """
    Returns a copy of df with a new column `group_id` containing the group number
    such that each group has min_size <= n <= max_size (as much as possible).
    Groups are formed by recursively splitting by postal code prefix, then merging small groups.

    Args:
        df (pd.DataFrame): Input DataFrame with a postal code column.
        code_col (str): Name of the column with postal codes (should be int or str, 5 digits).
        min_size (int): Minimum group size.
        max_size (int): Maximum group size.

    Returns:
        pd.DataFrame: Copy of df with a new 'group_id' column.
    """
    df['postal_clean'] = df[postal_code_col].astype(str).str.replace('-', '')
    work = df.copy()
    work["_code_str"] = work['postal_clean'].astype(str).str.zfill(5)

    accepted = {}                # {prefix: list[index]}
    small     = {}               # prefixy < min_size (to be merged later)

    # 1. Recursive splitting of large groups
    def split(prefix: str, idxs: list[int], depth: int):
        size = len(idxs)
        # a) Acceptable group
        if min_size <= size <= max_size or depth == 5:
            accepted[prefix] = idxs
            return
        # b) Too small – save for later merging
        if size < min_size:
            small[prefix] = idxs
            return
        # c) Too large – split deeper
        next_depth = depth + 1
        sub_prefixes = work.loc[idxs, "_code_str"].str[:next_depth]
        for sub_pref, sub_idxs in work.loc[idxs].groupby(sub_prefixes).groups.items():
            split(sub_pref, list(sub_idxs), next_depth)

    # Start with 2-digit prefixes
    for pref2, grp in work.groupby(work["_code_str"].str[:2]).groups.items():
        split(pref2, list(grp), depth=2)

    # 2. Merge small groups within the same 2-digit prefix
    buckets = defaultdict(list)          # {pref2: [(pref, idxs), ...]}
    for p, idxs in small.items():
        buckets[p[:2]].append((p, idxs))

    for pref2, lst in buckets.items():
        # Sort by prefix value for spatial proximity
        lst.sort(key=lambda x: int(x[0]))
        buf_idx, buf_pref = [], []
        for p, idxs in lst:
            buf_idx.extend(idxs)
            buf_pref.append(p)
            if len(buf_idx) >= min_size:
                # If > max_size, split into chunks
                while len(buf_idx) > max_size:
                    accepted[f"{pref2}_{len(accepted)}"] = buf_idx[:max_size]
                    buf_idx = buf_idx[max_size:]
                accepted["+".join(buf_pref)] = buf_idx
                buf_idx, buf_pref = [], []
        # Remainder < min_size – append to last group for this prefix
        if buf_idx:
            last_keys = [k for k in accepted.keys() if k.startswith(pref2)]
            if last_keys:
                last_key = last_keys[-1]
                accepted[last_key].extend(buf_idx)
            else:
                # If no group exists, create a new one
                accepted[f"{pref2}_rem"] = buf_idx

    # 3. Assign group numbers
    idx2gid = {}
    for gid, (_, lst) in enumerate(accepted.items()):
        for i in lst:
            idx2gid[i] = gid
    work["bucket"] = work.index.map(idx2gid)

    # 4. Remove groups with fewer than 4 members
    group_sizes = work.groupby('bucket').transform('count')['postal_clean']
    work = work[group_sizes > 3].copy()

    return work.drop(columns="_code_str")

In [86]:
df = add_bialek_postal_buckets(df_2025_r2, min_size=10, max_size=16)

In [88]:
df.head()

,valid_votes,polling_station_id,eligible_voters,ballots_cast,teryt_gmina,nawrocki,trzaskowski,postal_code,postal_clean,bucket
0,1178.0,1,1678.0,1187.0,20101.0,582.0,596.0,59-700,59700,1232
1,903.0,2,1269.0,911.0,20101.0,386.0,517.0,59-700,59700,1232
2,993.0,3,1358.0,996.0,20101.0,437.0,556.0,59-700,59700,1232
3,975.0,4,1354.0,984.0,20101.0,408.0,567.0,59-700,59700,1232
4,850.0,5,1195.0,856.0,20101.0,332.0,518.0,59-700,59700,1232


In [89]:
print_bucket_stats(df)

Total buckets: 2304
Buckets with 10–16 items: 1264 (54.9%)
Buckets with 6–30 items: 1917 (83.2%)


### IMPLEMENTACJA METOD ZAPROPONOWANYCH PRZEZ DR KONTKA

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5296441

2.3. Wykrywanie wartości odstających

Główna innowacja analityczna niniejszego badania polega na oszacowaniu potencjalnego wpływu
anormalnych komisji wyborczych na poziomie ogólnokrajowym. Aby to osiągnąć, w pierwszej
kolejności zidentyfikowano wartości odstające w czterech kategoriach nieprawidłowości:

In [ ]:
## let's do it with clustering proposed by Jakub Bialek

# df = add_bialek_postal_buckets(df, min_size=10, max_size=16)

In [136]:
keep_columns = ["teryt_gmina", "polling_station_id", "trzaskowski", "nawrocki", "postal_code"]

# ROUND 1 
# Step 1: Keep only selected columns
df_2025_r1 = df_2025_r1[keep_columns]
df_2025_r1 = df_2025_r1.dropna(subset=["teryt_gmina"])
# Step 2: Convert teryt_gmina to integer
df_2025_r1["teryt_gmina"] = df_2025_r1["teryt_gmina"].astype(int)

# ROUND 2
# Step 1: Keep only selected columns
df_2025_r2 = df_2025_r2[keep_columns]
df_2025_r2 = df_2025_r2.dropna(subset=["teryt_gmina"])
# Step 2: Convert teryt_gmina to integer
df_2025_r2["teryt_gmina"] = df_2025_r2["teryt_gmina"].astype(int)


In [137]:
# Join both rounds

# Step 1: Rename candidate columns in each round for clarity
df_2025_r1 = df_2025_r1.rename(columns={
    "trzaskowski": "trzaskowski_r1",
    "nawrocki": "nawrocki_r1"
})

df_2025_r2 = df_2025_r2.rename(columns={
    "trzaskowski": "trzaskowski_r2",
    "nawrocki": "nawrocki_r2"
})

# Step 2: Merge on teryt_gmina and polling_station_id
df_2025 = pd.merge(
    df_2025_r1,
    df_2025_r2,
    on=["teryt_gmina", "polling_station_id", "postal_code"],
    how="inner"  # or "outer"/"left" depending on need
)

In [138]:
# Add buckets
df_2025 = add_bialek_postal_buckets(df_2025)


In [139]:
df_2025.head()

,teryt_gmina,polling_station_id,trzaskowski_r1,nawrocki_r1,postal_code,trzaskowski_r2,nawrocki_r2,postal_clean,bucket
0,20101,1,361.0,287.0,59-700,596.0,582.0,59700,1567
1,20101,2,381.0,228.0,59-700,517.0,386.0,59700,1567
2,20101,3,356.0,241.0,59-700,556.0,437.0,59700,1567
3,20101,4,390.0,217.0,59-700,567.0,408.0,59700,1567
4,20101,5,343.0,202.0,59-700,518.0,332.0,59700,1567


In [111]:
cand_A = "trzaskowski"
cand_B = "nawrocki"

### 1. Nadmierne poparcie dla Karola Nawrockiego (względem mediany w ramach lokalnej grupy)

Za artykułem: w ramach każdej grupy obliczono mediany oraz odchylenia bezwzględne od mediany (MAD), dla każdej komisji obliczono wskaźnik odchylenia od mediany

![X minus median over MAD](./images/X_minus_median_over_MAD.png)

gdzie:

X - wynik w drugiej turze kandydata

mediana - mediana wyników kandydata w drugiej turze

MAD - odchelenie bezwzględne mediany kandydata w drugiej turze

z artykulu:

Dla każdej grupy komisji oraz dla dwóch pierwszych kategorii nieprawidłowości:

  • obliczono mediany oraz odchylenia bezwzględne od mediany (MAD);

  • dla każdej komisji obliczono tzw. współczynnik odchylenia odpornego (robust deviation score, oznaczony jako k_needed), wyrażający skalę odchylenia wyniku od mediany grupy w jednostkach MAD, według wzoru (powyzej)

Komisję oznaczano jako odstającą (outlier), jeśli spełniony był warunek: 𝑘𝑛𝑒𝑒𝑑𝑒𝑑 > 𝑘 gdzie k to
wartość progowa przyjęta w analizie, szczegółowo opisana w sekcji 2.4.

In [218]:
df = df_2025.copy()

In [219]:
# mediana w grupie
df[cand_A + '_median_r2'] = df.groupby('bucket')[cand_A + '_r2'].transform('median')
df[cand_B + '_median_r2'] = df.groupby('bucket')[cand_B + '_r2'].transform('median')

In [220]:
# MAD w grupie
from scipy.stats import median_abs_deviation
def mad(series):
    """Median Absolute Deviation"""
    return median_abs_deviation(series)

df[cand_A + "_MAD_r2"] = df.groupby("bucket")[cand_A + '_r2'].transform(mad)
df[cand_B + "_MAD_r2"] = df.groupby("bucket")[cand_B + '_r2'].transform(mad)

In [221]:
df[cand_A +'_k_score_1'] = (df[cand_A + '_r2'] - df[cand_A + '_median_r2'])/df[cand_A + '_MAD_r2']
df[cand_B +'_k_score_1'] = (df[cand_B + '_r2'] - df[cand_B + '_median_r2'])/df[cand_B + '_MAD_r2']

2.4 Przeliczenie wyników

Aby uwzględnić niepewność i wrażliwość zastosowanego podejścia, obliczenia przeprowadzono dla
trzech różnych progów detekcji wartości odstających: k > 2.0, k > 2.5 oraz k > 3.0, gdzie k oznacza
liczbę jednostek odchylenia bezwzględnego od mediany (MAD) względem mediany w grupie
lokalnej. Wyższe wartości k wyodrębniają jedynie najbardziej skrajne przypadki, zapewniając tym
samym konserwatywną estymację potencjalnego wpływu. Jednocześnie jednak ograniczają zdolność
metody do wychwytywania mniejszych, lecz wciąż istotnych odchyleń.

In [204]:
for k in [2.0, 2.5, 3.0]:
    count_A = sum(df[cand_A + '_k_score_1'] > k)
    count_B = sum(df[cand_B + '_k_score_1'] > k)
    print(f'k > {k}')
    print(f'{cand_A}: {count_A}')
    print(f'{cand_B}: {count_B}')
    print('---')

k > 2.0
trzaskowski: 4551
nawrocki: 3750
---
k > 2.5
trzaskowski: 3535
nawrocki: 2750
---
k > 3.0
trzaskowski: 2794
nawrocki: 2015
---


Wyniki:

Dla k=2, takich komisji, w których "za duże" poparcie ma Nawrocki jest 3762, a Trzaskowski 4554.

In [222]:
# Let's set k = 2
k = 2

In [223]:
# zapisz anomalie na korzyść kandydatów do późniejszego sumowania
df[cand_A + "_anomaly_1"] = df[kandydat_A +'_k_score_1'] > k
df[cand_B + "_anomaly_1"] = df[kandydat_B +'_k_score_1'] > k

In [224]:
# df.head()

df = df.drop(columns=[
    # "trzaskowski_median_r2",
    # "nawrocki_median_r2",
    "trzaskowski_MAD_r2",
    "nawrocki_MAD_r2",
    "trzaskowski_k_score_1",
    "nawrocki_k_score_1"
])

### 2. Nadmierny względny wzrost poparcia dla Karola Nawrockiego między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla Rafała Trzaskowskiego w tej samej grupie lokalnej;

za, JB: 

Nie podano wprost jak to było obliczone więc kolejno:

  1. Dla danego kandydata obliczam względny wzrost między pierwszą a drugą turą (dzieląc wynik z drugiej przez wynik z pierwszej)
  2. Następnie odnoszę go do wzrostu drugiego kandydata - liczę różnicę między względnymi wzrostami.
  3. Dalej tak jak w pierwszym typie anomalii - dla tych różnic liczę medianę grupy, MAD grupy oraz odchylenie k w komisji.

In [225]:
# wzgledny wzrost między pierwszą a drugą turą
df[cand_A + '_increase'] = df[cand_A + '_r2']/df[cand_A + '_r1']
df[cand_B + '_increase'] = df[cand_B + '_r2']/df[cand_B + '_r1']

In [226]:
# roznica wzglednego wzrostu miedzy kandydatami
df['relative_increase_diff_' + cand_A] =  df[cand_A + '_increase']  - df[cand_B + '_increase'] # wzrost A w porównaniu do B
df['relative_increase_diff_' + cand_B] =  df[cand_B + '_increase']  - df[cand_A + '_increase'] # wzrost B w porównaniu do A

In [227]:
# mediana i mad różnicy względnego wzrostu poparcia
df['relative_increase_diff_' + cand_A +'_median'] = df.groupby('bucket')['relative_increase_diff_' + cand_A].transform('median')
df['relative_increase_diff_' + cand_A + '_MAD'] = df.groupby('bucket')['relative_increase_diff_' + cand_A].transform(mad)

In [228]:
# mediana i mad różnicy względnego wzrostu poparcia
df['relative_increase_diff_' + cand_B +'_median'] = df.groupby('bucket')['relative_increase_diff_' + cand_B].transform('median')
df['relative_increase_diff_' + cand_B + '_MAD'] = df.groupby('bucket')['relative_increase_diff_' + cand_B].transform(mad)

In [229]:
df[cand_A +'_k_score_2'] = (df['relative_increase_diff_' + cand_A] - df['relative_increase_diff_' + cand_A +'_median'])/df['relative_increase_diff_' + cand_A + '_MAD']


In [230]:
df[cand_B +'_k_score_2'] = (df['relative_increase_diff_' + cand_B] - df['relative_increase_diff_' + cand_B +'_median'])/df['relative_increase_diff_' + cand_B + '_MAD']

In [231]:
for k in [2.0, 2.5, 3.0]:
    count_A = sum(df[cand_A + '_k_score_2'] > k)
    count_B = sum(df[cand_B + '_k_score_2'] > k)
    print(f'k > {k}')
    print(f'{cand_A}: {count_A}')
    print(f'{cand_B}: {count_B}')
    print('---')

k > 2.0
trzaskowski: 3552
nawrocki: 3127
---
k > 2.5
trzaskowski: 2666
nawrocki: 2229
---
k > 3.0
trzaskowski: 2106
nawrocki: 1669
---


In [232]:
# zapisz anomalie na korzyść kandydatów do późniejszego sumowania
df[cand_A + "_anomaly_2"] = df[kandydat_A +'_k_score_2'] > k
df[cand_B + "_anomaly_2"] = df[kandydat_B +'_k_score_2'] > k

In [233]:
df = df.drop(columns=[
    "trzaskowski_increase",
    "nawrocki_increase",
    "relative_increase_diff_trzaskowski",
    "relative_increase_diff_nawrocki",
    "relative_increase_diff_trzaskowski_median",
    "relative_increase_diff_trzaskowski_MAD",
    "relative_increase_diff_nawrocki_median",
    "relative_increase_diff_nawrocki_MAD",
    "trzaskowski_k_score_2",
    "nawrocki_k_score_2"
])

### 3. Komisje, w których Nawrocki uzyskał więcej głosów niż Trzaskowski w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę Trzaskowskiego;

  1. Sprawdzamy, w których grupach dany kandydat miał większą medianę
  2. Sumujemy komisje, w których wygrał kandydat A mimo, że większą medianę miał kandydat B i na odwrót.

In [234]:
# wieksza mediana w grupie
df['higher_median_' + cand_A] = (df[cand_A + '_median_r2'] >  df[cand_B + '_median_r2']).astype(bool)
df['higher_median_' + cand_B] = (df[cand_B + '_median_r2'] >  df[cand_A + '_median_r2']).astype(bool)

In [236]:
# na korzyść kandydat A, czyli większą medianę miał B, a więcej głosów dostał A.
cand_A, sum(df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']))

('trzaskowski', 2608)

In [238]:
cand_B, sum(df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']))

('nawrocki', 1843)

**WYNIKI**:

W grupach, w których większą medianę miał Nawrocki, było 2608 komisji, w których wyższy wynik uzyskał Trzaskowski.

W grupach, w których większą medianę miał Trzaskowski, było 1843 komisji, w których wyższy wyniki uzyskał Nawrocki.

Przykładowo:

W komisji 13 gdzie w drugiej turze głosowało.. 13 osób, Trzaskowski uzyskał większy wynik (8 do 5), mimo że w grupie obejmującej kod pocztowy 59-730 większą medianę miał Nawrocki (344 vs 158).

In [239]:
# anomalie na korzysc
df[cand_A + '_anomaly_3'] = df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']) 
df[cand_B + '_anomaly_3'] = df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']) 

In [241]:
df = df.drop(columns=[
    "trzaskowski_median_r2",
    "nawrocki_median_r2",
    "higher_median_trzaskowski",
    "higher_median_nawrocki"	
])

**Wątpliwości w tej metodologii**

Jak pisze Piotr Szulc:

https://danetyka.com/kontek-analiza-bledow/

Jedna z cech, jakie bada autor, jest nazwana “flip” i nie ma nic wspólnego z wyżej podaną standaryzacją i progami. Autor za anomalię uznaje każdy przypadek, w którym “Nawrocki wygrywa lokalnie, mimo że mediana wyników w grupie wskazuje przewagę Trzaskowskiego”. Załóżmy, że procenty poparcia dla Nawrockiego w danej grupie wynoszą: 45, 46, 47, 48, 49, 51, 52, 53, 54.

In [ ]:
# Dane: 9 komisji – Trzaskowski ma wyższą medianę, ale Nawrocki wygrywa w 4 komisjach
dummy_df = pd.DataFrame({
    'okręg': ['A'] * 9,
    'trzaskowski': [55, 54, 53, 52, 51, 49, 47, 46, 45],
    'nawrocki':    [45, 46, 47, 48, 49, 51, 52, 53, 54],
})

Mediana wynosi 49%, więc “mediana wyników w grupie wskazuje przewagę Trzaskowskiego”:

In [328]:
# Obliczenie median
trzaskowski_median = dummy_df['trzaskowski'].median()  # 51.0
nawrocki_median = dummy_df['nawrocki'].median()        # 49.0

# ale zeby być spójnym z poprzednią implementacją:

# mediana w grupie
dummy_df[cand_A + '_median'] = dummy_df.groupby('okręg')[cand_A].transform('median')
dummy_df[cand_B + '_median'] = dummy_df.groupby('okręg')[cand_B].transform('median')

dummy_df['higher_median_' + cand_A] = (dummy_df[cand_A + '_median'] >  dummy_df[cand_B + '_median']).astype(bool)
dummy_df['higher_median_' + cand_B] = (dummy_df[cand_B + '_median'] >  dummy_df[cand_A + '_median']).astype(bool)

a zatem te cztery komisje, w których Nawrockich otrzymał ponad 50% to anomalie, co oczywiście nie ma żadnego sensu. Ta cecha jest odpowiedzialna za ponad połowę (!) wskazań.

In [330]:
# na korzyść kandydat B, czyli większą medianę miał A, a więcej głosów dostał B.
cand_B, sum(dummy_df['higher_median_' + cand_A] & (dummy_df[cand_B] > dummy_df[cand_A]))

('nawrocki', 4)

In [332]:
# Flip: Nawrocki wygrywa, mimo że Trzaskowski miał wyższą medianę w grupie
dummy_df['flip_' + cand_B] = dummy_df['higher_median_' + cand_A] & (dummy_df[cand_B] > dummy_df[cand_A])

# Wyświetlenie flipów
print(dummy_df[[cand_A, cand_B, 'flip_' + cand_B]])
print(f"\nLiczba 'anomalii' według flip: {dummy_df['flip_' + cand_B].sum()} z {len(dummy_df)}")

   trzaskowski  nawrocki  flip_nawrocki
0           55        45          False
1           54        46          False
2           53        47          False
3           52        48          False
4           51        49          False
5           49        51           True
6           47        52           True
7           46        53           True
8           45        54           True

Liczba 'anomalii' według flip: 4 z 9


Na obronę dra Kontka muszę przyznać, że te „anomalie” nie zostałyby uwzględnione w jego właściwej analizie, ponieważ zastosowany przez niego próg istotności wynosi

_k = 2_. 

W naszym przykładzie, mimo że występują przypadki tzw. „flipów” (czyli lokalnej wygranej kandydata z niższą medianą), żaden z nich nie osiąga wartości 

_k_score_1 > 2_. 

Oznacza to, że różnice te nie zostałyby uznane za statystycznie istotne odchylenia w jego modelu i nie trafiłyby na listę „anomalii”.

In [333]:
# === Obliczanie MAD w grupie ===
def mad(series):
    return median_abs_deviation(series, scale='normal')  # spójne z klasyczną definicją

dummy_df[cand_A + '_MAD'] = dummy_df.groupby('okręg')[cand_A].transform(mad)
dummy_df[cand_B + '_MAD'] = dummy_df.groupby('okręg')[cand_B].transform(mad)

# Obliczanie k_score_1
dummy_df[cand_A + '_k_score_1'] = (dummy_df[cand_A] - dummy_df[cand_A + '_median']) / dummy_df[cand_A + '_MAD']
dummy_df[cand_B + '_k_score_1'] = (dummy_df[cand_B] - dummy_df[cand_B + '_median']) / dummy_df[cand_B + '_MAD']

# === Analiza k_score_1 względem progów ===
for k in [2.0, 2.5, 3.0]:
    count_A = (dummy_df[cand_A + '_k_score_1'] > k).sum()
    count_B = (dummy_df[cand_B + '_k_score_1'] > k).sum()
    print(f'\nk > {k}')
    print(f'{cand_A}: {count_A}')
    print(f'{cand_B}: {count_B}')
    print('---')


k > 2.0
trzaskowski: 0
nawrocki: 0
---

k > 2.5
trzaskowski: 0
nawrocki: 0
---

k > 3.0
trzaskowski: 0
nawrocki: 0
---


### 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [244]:
cand_A, sum(df[cand_A + '_r2']<df[cand_A + '_r1'])

('trzaskowski', 128)

W 128 komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

In [246]:
cand_B, sum(df[cand_B + '_r2']<df[cand_B + '_r1'])

('nawrocki', 112)

W 112 komisjach Nawrocki uzyskał mniej głosów w drugiej turze niż w pierwszej.


Przykładowe anomalie na korzyść Trzaskowskiego:

In [266]:
df[df[cand_B + '_r2'] < df[cand_B + '_r1']].sort_values(by=cand_B + '_r1', ascending=False).head()

,teryt_gmina,polling_station_id,trzaskowski_r1,nawrocki_r1,postal_code,trzaskowski_r2,nawrocki_r2,postal_clean,bucket,trzaskowski_anomaly_1,nawrocki_anomaly_1,trzaskowski_anomaly_2,nawrocki_anomaly_2,trzaskowski_anomaly_3,nawrocki_anomaly_3,trzaskowski_anomaly_4,nawrocki_anomaly_4,trzaskowski_sum_anomalies,nawrocki_sum_anomalies
12616,140706,1,105.0,285.0,26-910,467.0,193.0,26910,630,True,False,True,False,True,False,True,False,4,0
25866,261207,4,143.0,224.0,28-200,360.0,209.0,28200,662,True,False,True,False,True,False,True,False,4,0
5098,60903,4,89.0,174.0,23-100,260.0,163.0,23100,522,True,False,True,False,True,False,True,False,4,0
24825,260101,34,172.0,129.0,28-100,148.0,111.0,28100,657,False,False,False,False,True,False,True,True,2,1
2372,40102,9,120.0,105.0,87-720,164.0,85.0,87720,2197,False,False,True,False,True,False,True,False,3,0


In [250]:
# Anomalie na korzysc
df[cand_A + '_anomaly_4'] = df[cand_B + '_r2']<df[cand_B + '_r1']
df[cand_B + '_anomaly_4'] = df[cand_A + '_r2']<df[cand_A + '_r1']

To są rzeczywiście bardzo podejrzane przypadki i o takich przypadkach powinniśmy alarmować w pierwszej kolejności. Po pierwsze, ju na etapie wprowadzania do systemu, a po drugie do ewentualnej kontroli i ponownego liczenia glosów

## Sumowanie anomalii

### Na korzyść Trzaskowskiego

In [253]:
df[cand_A + '_sum_anomalies'] = df[[
    cand_A + '_anomaly_1', 
    cand_A + '_anomaly_2',
    cand_A + '_anomaly_3',
    cand_A + '_anomaly_4']].sum(axis=1)

In [ ]:
for number_of_anomalies in [1,2,3,4]:
    print(f"{number_of_anomalies} anomalies:")
    print(cand_A, sum(df[cand_A + '_sum_anomalies']>=number_of_anomalies))

1 anomalies:
trzaskowski 8161
2 anomalies:
trzaskowski 1179
3 anomalies:
trzaskowski 34
4 anomalies:
trzaskowski 3


In [259]:
# Komisje z wszystkimi czterma anomaliami
df[df[cand_A + '_sum_anomalies']>=4]

,teryt_gmina,polling_station_id,trzaskowski_r1,nawrocki_r1,postal_code,trzaskowski_r2,nawrocki_r2,postal_clean,bucket,trzaskowski_anomaly_1,nawrocki_anomaly_1,trzaskowski_anomaly_2,nawrocki_anomaly_2,trzaskowski_anomaly_3,nawrocki_anomaly_3,trzaskowski_anomaly_4,nawrocki_anomaly_4,trzaskowski_sum_anomalies
5098,60903,4,89.0,174.0,23-100,260.0,163.0,23100,522,True,False,True,False,True,False,True,False,4
12616,140706,1,105.0,285.0,26-910,467.0,193.0,26910,630,True,False,True,False,True,False,True,False,4
25866,261207,4,143.0,224.0,28-200,360.0,209.0,28200,662,True,False,True,False,True,False,True,False,4


### Na korzyść Nawrockiego

In [261]:
df[cand_B + '_sum_anomalies'] = df[[
    cand_B + '_anomaly_1', 
    cand_B + '_anomaly_2',
    cand_B + '_anomaly_3',
    cand_B + '_anomaly_4']].sum(axis=1)

In [264]:
for number_of_anomalies in [1,2,3,4]:
    print(f"{number_of_anomalies} anomalies:")
    print(cand_B, sum(df[cand_B + '_sum_anomalies']>=number_of_anomalies))

1 anomalies:
nawrocki 6871
2 anomalies:
nawrocki 483
3 anomalies:
nawrocki 34
4 anomalies:
nawrocki 2


In [263]:
# Komisje z 4 anomaliami, "widać Kraków"
df[df[cand_B + '_sum_anomalies']>=4]

,teryt_gmina,polling_station_id,trzaskowski_r1,nawrocki_r1,postal_code,trzaskowski_r2,nawrocki_r2,postal_clean,bucket,trzaskowski_anomaly_1,nawrocki_anomaly_1,trzaskowski_anomaly_2,nawrocki_anomaly_2,trzaskowski_anomaly_3,nawrocki_anomaly_3,trzaskowski_anomaly_4,nawrocki_anomaly_4,trzaskowski_sum_anomalies,nawrocki_sum_anomalies
11610,126101,95,550.0,218.0,31-346,540.0,1132.0,31346,691,False,True,False,True,False,True,False,True,0,4
17032,161105,9,311.0,107.0,47-100,223.0,416.0,47100,1408,False,True,False,True,False,True,False,True,0,4


### PONOWNIE POLICZONE GŁOSY


https://polskieradio24.pl/artykul/3543223,jakie-sa-wyniki-w-komisjach-w-ktorych-ponownie-przeliczono-glosy-sprawdzilismy

In [273]:
df_vote_recount = load_presidential_data("2025", "2")
df_vote_recount = process_presidential_df(df_vote_recount, "2025", final_cols=["Gmina", "Województwo", "Siedziba"])

In [284]:
# Example: values from recount
target_nawrocki = 1132
target_trzaskowski = 540

# Find records that match these values exactly
matching_stations = df_vote_recount[
    (df_vote_recount["nawrocki"] == target_nawrocki) &
    (df_vote_recount["trzaskowski"] == target_trzaskowski)
]

print("Matching polling stations after recount:")
print(matching_stations.T)

Matching polling stations after recount:
                                                                11610
valid_votes                                                    1672.0
polling_station_id                                                 95
eligible_voters                                                1980.0
ballots_cast                                                   1684.0
teryt_gmina                                                  126101.0
nawrocki                                                       1132.0
trzaskowski                                                     540.0
postal_code                                                    31-346
Gmina                                                       m. Kraków
Województwo                                               małopolskie
Siedziba            Zespół Szkolno-Przedszkolny Nr 14, ul. Stawowa...


In [299]:
# --- Step 1: Create the recount dataset with both old and new values ---
recounts = [
    {"polling_station_id": 95,  "valid_votes": 1672, "old_nawrocki": 1132, "old_trzaskowski": 540,  "new_nawrocki": 540,  "new_trzaskowski": 1132},
    {"polling_station_id": 3,   "valid_votes": 1015, "old_nawrocki": 637,  "old_trzaskowski": 378,  "new_nawrocki": 377,  "new_trzaskowski": 638},
    {"polling_station_id": 13,  "valid_votes": 974,  "old_nawrocki": 611,  "old_trzaskowski": 363,  "new_nawrocki": 364,  "new_trzaskowski": 611},
    {"polling_station_id": 9,   "valid_votes": 639,  "old_nawrocki": 416,  "old_trzaskowski": 223,  "new_nawrocki": 223,  "new_trzaskowski": 416},
    {"polling_station_id": 25,  "valid_votes": 828,  "old_nawrocki": 504,  "old_trzaskowski": 324,  "new_nawrocki": 324,  "new_trzaskowski": 504},
    {"polling_station_id": 17,  "valid_votes": 931,  "old_nawrocki": 585,  "old_trzaskowski": 346,  "new_nawrocki": 344,  "new_trzaskowski": 585},
    {"polling_station_id": 30,  "valid_votes": 959,  "old_nawrocki": 610,  "old_trzaskowski": 349,  "new_nawrocki": 450,  "new_trzaskowski": 509},
    {"polling_station_id": 61,  "valid_votes": 1819, "old_nawrocki": 1048, "old_trzaskowski": 771,  "new_nawrocki": 771,  "new_trzaskowski": 1049},
    {"polling_station_id": 10,  "valid_votes": 330,  "old_nawrocki": 217,  "old_trzaskowski": 113,  "new_nawrocki": 317,  "new_trzaskowski": 363},
    {"polling_station_id": 53,  "valid_votes": 1458, "old_nawrocki": 628,  "old_trzaskowski": 830,  "new_nawrocki": 627,  "new_trzaskowski": 828},
    {"polling_station_id": 35,  "valid_votes": 928,  "old_nawrocki": 581,  "old_trzaskowski": 347,  "new_nawrocki": 347,  "new_trzaskowski": 581},
    {"polling_station_id": 6,   "valid_votes": 706,  "old_nawrocki": 368,  "old_trzaskowski": 338,  "new_nawrocki": 278,  "new_trzaskowski": 428},
    {"polling_station_id": 4,   "valid_votes": 797,  "old_nawrocki": 466,  "old_trzaskowski": 331,  "new_nawrocki": 331,  "new_trzaskowski": 466},
    {"polling_station_id": 4,   "valid_votes": 569,  "old_nawrocki": 209,  "old_trzaskowski": 360,  "new_nawrocki": 360,  "new_trzaskowski": 209},  # Staszów
    {"polling_station_id": 1,   "valid_votes": 660,  "old_nawrocki": 193,  "old_trzaskowski": 467,  "new_nawrocki": 468,  "new_trzaskowski": 192},  # Magnuszew
    {"polling_station_id": 113, "valid_votes": 1910, "old_nawrocki": 136,  "old_trzaskowski": 1774, "new_nawrocki": 296,  "new_trzaskowski": 1611},
    {"polling_station_id": 20,  "valid_votes": 1225, "old_nawrocki": 543,  "old_trzaskowski": 682,  "new_nawrocki": 542,  "new_trzaskowski": 683},
]

recount_df = pd.DataFrame(recounts)

# --- Step 2: Merge on 4 fields for exact match ---
df_affected_polling_stations = df_vote_recount.merge(
    recount_df,
    how="inner",
    left_on=["polling_station_id", "valid_votes", "nawrocki", "trzaskowski"],
    right_on=["polling_station_id", "valid_votes", "old_nawrocki", "old_trzaskowski"]
)

# --- Step 3: Output ---
# print("✅ Matches with recount corrections:")
df_affected_polling_stations[["teryt_gmina", "polling_station_id", "valid_votes", "nawrocki", "new_nawrocki",
               "trzaskowski", "new_trzaskowski"]].head(17)




,teryt_gmina,polling_station_id,valid_votes,nawrocki,new_nawrocki,trzaskowski,new_trzaskowski
0,20701.0,6,706.0,368.0,278,338.0,428
1,41804.0,4,797.0,466.0,331,331.0,466
2,46201.0,25,828.0,504.0,324,324.0,504
3,121611.0,10,330.0,217.0,317,113.0,363
4,126101.0,95,1672.0,1132.0,540,540.0,1132
5,140706.0,1,660.0,193.0,468,467.0,192
6,141201.0,13,974.0,611.0,364,363.0,611
7,146505.0,113,1910.0,136.0,296,1774.0,1611
8,160803.0,3,1015.0,637.0,377,378.0,638
9,161105.0,9,639.0,416.0,223,223.0,416


In [301]:
# Podsumowanie weryfikacji wyników wyborów 17 komisji

len(df_affected_polling_stations)

17